# Capstone: Pre-processing and Training Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
import sys

## Import of csv's created in nfl_capstone_EDA

In [2]:
year = pd.read_csv('teamstarterdraft.csv')
yearAV = pd.read_csv('teamstarterdraftAV.csv')
week = pd.read_csv('weekstarterdraft.csv')
weekAV = pd.read_csv('weekstarterdraftAV.csv')

### Creation of 2 additional dataframes that do not include the categorical variables of coach, offcoor, defcoor, off scheme, and def align.  If these do not add much to the variance, they are just preventing tree based algorithms from functioning well, as the number of dummy columns is large

In [3]:
yearnocoach = year.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])
yearnocoachAV = yearAV.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])

## Create pipeline for Regression analysis

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error



yearAV_X = yearAV.drop('DraftTeamSelection', axis=1)
yearAV_y = yearAV['DraftTeamSelection']

week_X = week.drop('DraftTeamSelection', axis=1)
week_y = week['DraftTeamSelection']

weekAV_X = weekAV.drop('DraftTeamSelection', axis=1)
weekAV_y = weekAV['DraftTeamSelection']

dfdict = {'year':year, 'yearAV':yearAV}
#, 'week':week,'weekAV':weekAV

dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftTeamSelection', axis=1)
        y = df['DraftTeamSelection']
        categorical_features = list(year.select_dtypes(include=['category', object]).columns)
        categorical_transformer = OneHotEncoder(sparse=True, handle_unknown='error', drop='first')

        numeric_features = list(year.select_dtypes(include=['int', 'float']).columns)
        numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
        ])

        preprocessor = ColumnTransformer(
        transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        reg = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', LinearRegression())])
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        R2 = r2_score(y_test, y_pred)
        MSE = mean_squared_error(y_test, y_pred)
        MAE = mean_absolute_error(y_test, y_pred)
        results = pd.DataFrame({"R2": R2, 'MSE':MSE, 'MAE':MAE}, index=[key])
        dfresults = dfresults.append(results)
print(dfresults)

              R2       MSE       MAE
year   -0.145976  8.289298  2.425415
yearAV -0.135581  7.660749  2.334789


model score: -0.136
